# Data Analysis using Machine Learning techniques - 2020105742 한지훈 

# 0. Exploratory Data Analysis

## 프로젝트 개요

[주제]: 저출산 문제

[목적]: 

1. 실제 데이터를 통해 대한민국 출산율의 심각성을 알리자.

<img src="photos/1.png" width="500"/>
출처: https://www.etoday.co.kr/news/view/2356928

최근 출산율이 0.72로 역대 최저치를 기록했다는 기사들이 쏟아진다. 하지만 이정도의 언급이 있을 뿐이지, 전세계 국가들 출산율과 비교했을 때 어느 정도 위치에 있는지, 그래서 얼마나 심각한지에 대해서는 언급이 많이 되어있지 않다. 이 부분을 확실히 하기 위해서 Outlier 개념을 도입하여, z-score과 iqr 계산을 통해 "한국의 출산율은 전세계 기준으로, outlier라 불릴 정도로 동떨어져 있나(심각한가)?"에 대한 답을 하고자 한다.

2. 출산율과 연관있는 의외의 factor들에는 무엇이 있을까?

<img src="photos/2.png" width="500"/>
출처: https://www.betterfuture.go.kr/front/policySpace/policyReferenceDetail.do?articleId=23&listLen=10&searchKeyword=&position=M

2024 정부의 저출산 대책을 봐도, 대부분 비용 지원 차원(경제적 지원)의 방식으로 이루어진다. supervised, unsupervised 방식을 통해 경제적인 지원 말고, 출산율을 높일 수 있는 다른 factor들을 가려냄으로써, 정부가 좀 더 폭넓은 방향으로 정책을 만들도록 할 계획이다. 

3. 현재 시행중인 경제적 지원 중심 정책이 효과가 있는가?

현재 시행 중인 경제적 지원 중심의 정책도 효과가 있는지 검증해야 지원량을 줄일지 늘릴지 등의 판단을 할 수 있을 것이다. t-test를 통해 경제력이 출산율에 유의미한 영향력을 미치는지 확인해보고자 한다.

-> 이 사실들을 정부, 관련 기관, 국민들에게 알려 출산율 문제 해결에 기여하고자 한다.

## 사용할 Data 종류, 설명

1. List of countries by total fertility rate : https://tradingeconomics.com/country-list/fertility-rate-total-births-per-woman-wb-data.html

2. List of countries by GDP : https://www.worldometers.info/gdp/gdp-by-country/

3. List of countries by obesity rate : https://renewbariatrics.com/obesity-rank-by-countries/

4. List of countries by sex ratio : https://statisticstimes.com/demographics/countries-by-sex-ratio.php

5. List of countries by Literacy Rates: https://www.uscareerinstitute.edu/blog/which-countries-have-the-highest-and-lowest-literacy-rates

6. List of countries by Healthcare : https://wisevoter.com/country-rankings/best-healthcare-in-the-world/

7. List of countries by dependency ratio : https://www.indexmundi.com/facts/indicators/SP.POP.DPND.OL/rankings

8. List of countries by female labor participation rate : https://www.indexmundi.com/facts/indicators/SL.TLF.CACT.FE.ZS/rankings

# 1. Data Pre-processing

## Data 웹크롤링 하기

먼저 1. List of countries by total fertility rate(국가별 출산율)을 제공하는 페이지에서 listly를 통해 웹크롤링을 하여 개요를 확인했다.

원본:
<img src="photos/3.png" width="600"/>

In [1]:
import pandas as pd

fertility_rate_raw = pd.read_excel('raw_files/fertility_rate_raw.xlsx')
fertility_rate_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3
0,Aruba,1.18,%
1,Afghanistan,4.64,%
2,Angola,5.30,%


확인 결과 국가별 이름과 필요 attribute인 출산율이 모두 있다. 전처리 작업으로는 필요없는 %(LABEL-3)를 삭제해주고 label 이름들을 다시 붙여주면 될 것이다.

In [2]:
fertility_rate_raw = fertility_rate_raw.drop(labels=['LABEL-3'],axis=1)
fertility_rate_raw.columns = ['Name', 'Fertility_rate'] 

fertility_rate_raw.head(3)

,Name,Fertility_rate
0,Aruba,1.18
1,Afghanistan,4.64
2,Angola,5.30


다음으로 나머지 애트리뷰트들을 제공하는 페이지들에서 listly를 통해 각각 웹크롤링을 하여 개요를 확인했다.

2. GDP
<img src="photos/4.png" width="500"/>

In [3]:
GDP_raw = pd.read_excel('raw_files/GDP_raw.xlsx')
GDP_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5,LABEL-6,LABEL-7,LABEL-8
0,1,United States,"$25,462,700,000,000",$25.463 trillion,2.06%,"338,289,857","$75,269",25.32%
1,2,China,"$17,963,200,000,000",$17.963 trillion,2.99%,"1,425,887,337","$12,598",17.86%
2,3,Japan,"$4,231,140,000,000",$4.231 trillion,1.03%,"123,951,692","$34,135",4.21%


원본에서 GDP per capita(1인당 GDP)가 유의미하다고 판단하여 해당 애트리뷰트(LABEL-7)만 남겼다. 또한 실수값으로 활용하기 위해서 앞의 $ 표시를 삭제하고, label 이름들을 다시 붙여주었다.

In [4]:
GDP = []
for i in range(len(GDP_raw['LABEL-7'])):
    f = GDP_raw['LABEL-7'][i][1:]
    f = float(f.replace(',', ''))
    GDP.append(f)

GDP_raw.insert(3, "GDP", GDP)

GDP_raw = GDP_raw.drop(labels=['LABEL-1','LABEL-3','LABEL-4','LABEL-5','LABEL-6','LABEL-7','LABEL-8'],axis=1)
GDP_raw.columns = ['Name', 'GDP'] 

GDP_raw.head(3)

,Name,GDP
0,United States,75269.0
1,China,12598.0
2,Japan,34135.0


3. obesity rate
<img src="photos/5.png" width="500"/>

In [5]:
obesity_rate_raw= pd.read_excel('raw_files/obesity_rate_raw.xlsx')
obesity_rate_raw.head()

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5,LABEL-6
0,1,Cook Islands,"17,380","11,742","5,965",50.80%
1,2,Palau,"18,055","12,198","5,806",47.60%
2,3,Nauru,"12,668","8,559","3,903",45.60%
3,4,Samoa,"222,382","150,241","65,205",43.40%
4,5,Tonga,"106,858","72,193","31,260",43.30%


비만인 사람들의 비만율(LABEL-6)만을 남기고, float값으로 활용하기 위해 %를 제거해주었다.

In [6]:
obesity_rate = []
for i in range(len(obesity_rate_raw['LABEL-6'])):
    f = obesity_rate_raw['LABEL-6'][i][:-1]
    f = float(f)
    obesity_rate.append(f)

obesity_rate_raw.insert(3, "obesity_rate", obesity_rate)

obesity_rate_raw = obesity_rate_raw.drop(labels=['LABEL-1','LABEL-3','LABEL-4','LABEL-5','LABEL-6',],axis=1)
obesity_rate_raw.columns = ['Name', 'obesity_rate'] 

obesity_rate_raw.head(3)

,Name,obesity_rate
0,Cook Islands,50.8
1,Palau,47.6
2,Nauru,45.6


4. sex ratio
<img src="photos/6.png" width="500"/>

In [7]:
sex_ratio_raw = pd.read_excel('raw_files/sex_ratio_raw.xlsx')
sex_ratio_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5,LABEL-6,LABEL-7,LABEL-8
0,Armenia,81.799,1,108.3,110.161,82.066,52.008,Asia
1,Guadeloupe,82.366,2,103.3,103.274,81.275,70.641,North America
2,Ukraine,84.353,3,106.2,106.811,93.979,48.510,Europe


여성 100명당 남성의 수를 나타내는 LABEL-2를 제외하고 모두 제거하였다.

In [8]:
sex_ratio_raw = sex_ratio_raw.drop(labels=['LABEL-3','LABEL-4','LABEL-5','LABEL-6','LABEL-7','LABEL-8'],axis=1)
sex_ratio_raw.columns = ['Name', 'sex_ratio'] 

sex_ratio_raw.head(3)

,Name,sex_ratio
0,Armenia,81.799
1,Guadeloupe,82.366
2,Ukraine,84.353


5. literacy rate
<img src="photos/7.png" width="500"/>

In [9]:
literacy_rate_raw = pd.read_excel('raw_files/literacy_rate_raw.xlsx')
literacy_rate_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4
0,Country,NaN,Literacy rate,NaN
1,NaN,Andorra,NaN,100.0
2,NaN,Finland,NaN,100.0


국가별 문맹률을 나타내는 LABEL-4을 제외하고 모두 제거하였다. 또한 첫 열은 attribute 이름을 가지고 있어서 제거하고 따로 이름을 지었다.

In [10]:
literacy_rate_raw = literacy_rate_raw.drop(labels=['LABEL-1','LABEL-3'],axis=1)
literacy_rate_raw = literacy_rate_raw.drop([0], axis=0)
literacy_rate_raw.columns = ['Name', 'literacy_rate'] 

literacy_rate_raw.head(3)

,Name,literacy_rate
1,Andorra,100.0
2,Finland,100.0
3,Greenland,100.0


6. healthcare
<img src="photos/8.png" width="500"/>

In [11]:
healthcare_raw = pd.read_excel('raw_files/healthcare_raw.xlsx')
healthcare_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5
0,1,Belgium,83.8,80.6,NaN
1,2,Japan,83.2,86.6,NaN
2,3,Sweden,83.1,82.1,24%


의료수준을 나타내는 health score(LABEL-4)를 제외하고 모두 제거하였다.

In [12]:
healthcare_raw = healthcare_raw.drop(labels=['LABEL-1','LABEL-3', 'LABEL-5'],axis=1)
healthcare_raw.columns = ['Name', 'healthcare'] 

healthcare_raw.head(3)

,Name,healthcare
0,Belgium,80.6
1,Japan,86.6
2,Sweden,82.1


7. dependency ratio
<img src="photos/9.png" width="200"/>

In [13]:
dependency_ratio_raw = pd.read_excel('raw_files/dependency_ratio_raw.xlsx')
dependency_ratio_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5,LABEL-6,LABEL-7,LABEL-8
0,Rank,NaN,Country,NaN,Value,NaN,Year,NaN
1,NaN,1.0,NaN,Japan,NaN,48.01,NaN,2020.0
2,NaN,2.0,NaN,Finland,NaN,36.63,NaN,2020.0


dependency ratio를 나타내는 LABEL-6를 제외하고 모두 제거하였다. 또한 첫 열은 attribute 이름을 가지고 있어서 제거하고 따로 이름을 지었다.

In [14]:
dependency_ratio_raw = dependency_ratio_raw.drop(labels=['LABEL-1','LABEL-2','LABEL-3','LABEL-5','LABEL-7','LABEL-8'],axis=1)
dependency_ratio_raw = dependency_ratio_raw.drop([0], axis=0)
dependency_ratio_raw.columns = ['Name', 'dependency_ratio'] 

dependency_ratio_raw.head(3)

,Name,dependency_ratio
1,Japan,48.01
2,Finland,36.63
3,Italy,36.57


8. female labor participation rate
<img src="photos/10.png" width="400"/>

In [15]:
female_labor_raw = pd.read_excel('raw_files/female_labor_raw.xlsx')
female_labor_raw.head(3)

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5,LABEL-6,LABEL-7,LABEL-8
0,Rank,NaN,Country,NaN,Value,NaN,Year,NaN
1,NaN,1.0,NaN,Solomon Islands,NaN,83.06,NaN,2021.0
2,NaN,2.0,NaN,Rwanda,NaN,82.50,NaN,2021.0


female labor participation rate를 나타내는 LABEL-6를 제외하고 모두 제거하였다. 또한 첫 열은 attribute 이름을 가지고 있어서 제거하고 따로 이름을 지었다.

In [16]:
female_labor_raw = female_labor_raw.drop(labels=['LABEL-1','LABEL-2','LABEL-3','LABEL-5','LABEL-7','LABEL-8'],axis=1)
female_labor_raw = female_labor_raw.drop([0], axis=0)
female_labor_raw.columns = ['Name', 'female_labor'] 

female_labor_raw.head(3)

,Name,female_labor
1,Solomon Islands,83.06
2,Rwanda,82.50
3,Madagascar,81.53


## Data들 merge

분석을 위해서 크롤링한 8개의 dataset을 하나로 합치는 작업을 할 것이다. 그런데 8개 dataset들을 합치는 과정에서 Null값들이 반드시 발생하게 된다. 그 이유는 8개 dataset의 출처가 각기 다르고, 포함된 국가들이 다 조금씩 다르기 때문이다. (ex) 국가 "Aruba"는 Fertility_rate에는 존재하지만, GDP에는 존재하지 않음) outer join을 하면 그 사실을 확인할 수 있다.

In [17]:
a=fertility_rate_raw
b=GDP_raw
c=obesity_rate_raw
d=sex_ratio_raw
e=literacy_rate_raw
f=healthcare_raw
g=dependency_ratio_raw
h=female_labor_raw

In [18]:
x = pd.merge(left = a , right = b, how = "outer", on = "Name")
x = pd.merge(left = x , right = c, how = "outer", on = "Name")
x = pd.merge(left = x , right = d, how = "outer", on = "Name")
x = pd.merge(left = x , right = e, how = "outer", on = "Name")
x = pd.merge(left = x , right = f, how = "outer", on = "Name")
x = pd.merge(left = x , right = g, how = "outer", on = "Name")
x = pd.merge(left = x , right = h, how = "outer", on = "Name")
x.head(10)

,Name,Fertility_rate,GDP,obesity_rate,sex_ratio,literacy_rate,healthcare,dependency_ratio,female_labor
0,Aruba,1.18,NaN,NaN,88.891,97.99,NaN,NaN,NaN
1,Afghanistan,4.64,NaN,2.9,101.847,37.27,51.0,4.77,14.85
2,Angola,5.30,2999.0,10.2,97.811,72.28,50.3,4.26,73.97
3,Albania,1.39,6643.0,17.6,99.032,98.45,73.8,21.61,50.73
4,United Arab Emirates,1.46,53758.0,37.2,222.664,98.13,77.8,1.51,46.54
5,Argentina,1.88,13904.0,26.3,98.088,98.09,77.2,17.71,50.01
6,Armenia,1.58,7014.0,19.5,81.799,99.79,74.5,17.52,42.66
7,Antigua and Barbuda,1.58,18745.0,30.9,91.536,98.95,NaN,13.56,NaN
8,Australia,1.70,64003.0,28.6,98.674,99.00,80.2,25.14,61.06
9,Austria,1.48,52732.0,18.4,97.135,98.00,79.6,28.93,55.51


## Missing data processing

Null값들에 대처하기 위해 3가지 방법을 고민해볼 수 있다.

1. Retention: 만일 결측치가 그 자체로 의미가 있다면, 남겨두는 것도 방법일 것이다. 하지만 여기서 결측치가 발생한 이유는 단순히 data간 국가의 집합이 일치하지 않아서 발생한 것으로, 의미가 없기 때문에 남겨둘 이유가 없다.

2. Imputation
    - cold deck: 해당 결측치들에 대해서, 외부 도메인 지식으로 채우는 것은 어려움이 있다. 예를 들어 도메인 지식으로 아프가니스탄의 GDP를 알기는 어렵다.
    - hot deck: 국가 지표는 다른 국가 지표와 연관된 값이 아니기 때문에 mean, fill, KNN 등을 사용해서 채우기에는 어려움이 있다.    
    
3. Deletion: 앞선 2개 방법의 사용이 어렵고, 또한 결측치를 제거함으로써 결정을 더 빠르게 할 수 있기 때문에 삭제하기로 하였다. drop의 방법으로는 inner join을 함으로써, 7개의 지표를 모두 가지고 있는 국가만 남게 하였다.

In [19]:
x = pd.merge(left = a , right = b, how = "inner", on = "Name")
x = pd.merge(left = x , right = c, how = "inner", on = "Name")
x = pd.merge(left = x , right = d, how = "inner", on = "Name")
x = pd.merge(left = x , right = e, how = "inner", on = "Name")
x = pd.merge(left = x , right = f, how = "inner", on = "Name")
x = pd.merge(left = x , right = g, how = "inner", on = "Name")
x = pd.merge(left = x , right = h, how = "inner", on = "Name")
x

,Name,Fertility_rate,GDP,obesity_rate,sex_ratio,literacy_rate,healthcare,dependency_ratio,female_labor
0,Angola,5.30,2999.0,10.2,97.811,72.28,50.3,4.26,73.97
1,Albania,1.39,6643.0,17.6,99.032,98.45,73.8,21.61,50.73
2,United Arab Emirates,1.46,53758.0,37.2,222.664,98.13,77.8,1.51,46.54
3,Argentina,1.88,13904.0,26.3,98.088,98.09,77.2,17.71,50.01
4,Armenia,1.58,7014.0,19.5,81.799,99.79,74.5,17.52,42.66
...,...,...,...,...,...,...,...,...,...
124,Uruguay,1.49,20795.0,26.7,94.116,98.77,78.1,23.36,54.78
125,Uzbekistan,3.17,2322.0,15.5,100.215,100.00,76.3,7.21,44.90
126,South Africa,2.37,6776.0,26.8,95.281,95.02,56.6,8.39,46.21
127,Zambia,4.31,1488.0,8.9,97.367,87.50,59.8,3.96,69.23


join 결과, 129개 국가에 대해 이름 제외 8개의 attribute로, 최종적으로 도합 129 * 8 = 1032개의 data point를 수집하였다. 최종 파일을 엑셀파일로 변환하여 활용하려 한다.

In [20]:
raw_data = x
raw_data.to_excel("raw_data.xlsx")

In [21]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              129 non-null    object 
 1   Fertility_rate    129 non-null    float64
 2   GDP               129 non-null    float64
 3   obesity_rate      129 non-null    float64
 4   sex_ratio         129 non-null    float64
 5   literacy_rate     129 non-null    float64
 6   healthcare        129 non-null    float64
 7   dependency_ratio  129 non-null    float64
 8   female_labor      129 non-null    float64
dtypes: float64(8), object(1)
memory usage: 10.1+ KB


## Outlier processing(한국)

해당 dataset에서 outlier는 의미가 있다. outlier가 있다고 해도, 오류가 아니고 그 국가의 다양한 요인들 때문에 다른 국가들과 출산율, GDP 등의 차이가 많이 나게 되는 것이기 때문에 삭제하거나 수정하지 않고, Retention(그냥 두기)가 좋아보인다. 모델에게 유의미한 정보를 전달할 수 있기 때문이다.

다만, 개요에서 언급한 것 처럼, 한국의 출산율이 "outlier"이라고 불릴 정도로 적은지, 즉 심각한 상황인지를 검증해보고자 한다. 출산율만 봐도 되기 때문에 fertility_rate_raw를 사용하였다.

In [22]:
fertility_rate_raw.loc[fertility_rate_raw['Name'] == 'South Korea']

,Name,Fertility_rate
100,South Korea,0.808


In [23]:
fertility_rate_raw.describe()

,Fertility_rate
count,206.000000
mean,2.511733
std,1.249751
min,0.772000
25%,1.580000
50%,2.030000
75%,3.235000
max,6.820000


lower bound Q1 = 1.58, upper bound Q3 = 3.235임을 알 수 있다. 한국의 출산율 0.808은 이 범위에 속하지 않기 때문에, outlier이라고 가정할 수 있다. 게다가 꽤 차이가 많이 나기 때문에 심각한 지표라고 볼 수 있다. 

# 2. Regression

출산율과 연관있는 의외의 factor 찾기

In [24]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score